In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

In [30]:
def clear_gpu():
    import gc, torch
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

# Classification

In [ ]:
import json
file_path = "/kaggle/input/classification/classification_instruction_data_en.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    data = [json.loads(line.strip()) for line in f]

In [ ]:
valid_labels = {"negative", "positive", "neutral"}
data = [sample for sample in data if sample['output'].strip().lower() in valid_labels]

In [ ]:
import re

def extract_answer(text):
    matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', text)
    if matches:
        return matches[-1]
    return None

In [ ]:
def extract_target(text):
  for line in text.splitlines()[::-1]:
    if line.strip().startswith("####"):
      return line.strip().replace("####","").strip()
  return None

In [ ]:
def split_cot_and_answer(output_text):
    lines = output_text.strip().splitlines()
    cot_lines = [line for line in lines if not line.strip().startswith("####")]
    answer = extract_target(output_text)
    return "\n".join(cot_lines), answer

**Zero-shot**

In [ ]:
true_label = []
for i in range(200):
    true_label.append(data[i]['output'])

In [ ]:
acc = 0
true_results = []
for i in range(200):
  zero_shot_prompt = "Classify the sentiment of this sentence as positive, negative, or neutral:\n{text}"
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(data[i]['output'])
  print(de)
  if de == data[i]['output']:
    acc = acc+1
    true_results.append(de)
print(acc/200)

In [ ]:
from collections import Counter

counts = Counter(true_results)

print(counts)


In [ ]:
counts = Counter(true_label)

print(counts)

Zero-shot + CoT (ReAct)

In [ ]:
acc = 0
true_results = []
results = []
for i in range(200):
  zero_shot_prompt = f"""
  You are a classifier assistant.Classify the sentiment of this sentence as positive, negative, or neutral:\n{data[i]['input']}. \n
  A: Let's think step by step. The answer is ?"""
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de == data[i]['output']:
    acc = acc+1
    true_results.append(de)
  results.append(de)
print(acc/200)
print(Counter(true_results))
print(Counter(results))

In [ ]:
acc = 0
true_results = []
results = []
for i in range(200):
  zero_shot_prompt = f"""
  You are a classifier assistant.Classify the sentiment of this sentence as positive, negative, or neutral:\n{data[i]['input']}.
  Recorrect with other LLM models. \n
  A: Let's think step by step. The sentiment is """
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de == data[i]['output']:
    acc = acc+1
    true_results.append(de)
  results.append(de)
print(acc/200)
print(Counter(true_results))
print(Counter(results))

Zero-shot + ToT

In [ ]:
import re

def extract_labels_from_thoughts(text):
    matches = re.findall(r'\b(positive|negative|neutral)\b', text.lower())
    return list(set(matches))  # loại trùng nếu cần


In [ ]:
import re

def clean_label(text):
    match = re.search(r"\b(positive|negative|neutral)\b", text.lower())
    return match.group(1) if match else "UNKNOWN"


In [ ]:
def generate_strategy(prompt_base, strategy_index):
    prompt = prompt_base + f"Strategy {strategy_index + 1}:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
        **inputs,
        max_length=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
# acc = 0
# for i in range(200):
#     prompt_base = zero_shot_ToT(data[i])
#     strategies = [generate_strategy(prompt_base, j) for j in range(3)]
#     labels = extract_labels_from_thoughts("\n".join(strategies))
#     if labels:
#         prompt = followup_prompt(", ".join(labels))
#         input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
#         output = model.generate(
#             **input_ids,
#             max_length=20,
#             num_beams=5,
#             no_repeat_ngram_size=2,
#             early_stopping=True
#         )
#         final_label = tokenizer.decode(output[0], skip_special_tokens=True)
#         final_label = clean_label(final_label)
#         if final_label == data[i]['output']:
#             acc = acc+1
# print(acc/200)

In [ ]:
# acc = 0
# for i in range(200):
#     prompt_base = zero_shot_ToT(data[i])
#     strategies = [generate_strategy(prompt_base, j) for j in range(5)]
#     labels = extract_labels_from_thoughts("\n".join(strategies))
#     if labels:
#         prompt = followup_prompt(", ".join(labels))
#         input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
#         output = model.generate(
#             **input_ids,
#             max_length=20,
#             num_beams=5,
#             no_repeat_ngram_size=2,
#             early_stopping=True
#         )
#         final_label = tokenizer.decode(output[0], skip_special_tokens=True)
#         final_label = clean_label(final_label)
#         if final_label == data[i]['output']:
#             acc = acc+1
# print(acc/200)

In [ ]:
# acc = 0
# for i in range(200):
#     prompt_base = zero_shot_ToT(data[i])
#     strategies = [generate_strategy(prompt_base, j) for j in range(10)]
#     labels = extract_labels_from_thoughts("\n".join(strategies))
#     if labels:
#         prompt = followup_prompt(", ".join(labels))
#         input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
#         output = model.generate(
#             **input_ids,
#             max_length=20,
#             num_beams=5,
#             no_repeat_ngram_size=2,
#             early_stopping=True
#         )
#         final_label = tokenizer.decode(output[0], skip_special_tokens=True)
#         final_label = clean_label(final_label)
#         if final_label == data[i]['output']:
#             acc = acc+1
# print(acc/200)

Zero-shot + CoT + SC

In [ ]:
from collections import Counter
def self_consistency(prompt, num_samples=5, max_len=50):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
        output = model.generate(
            **inputs,
            max_length=max_len,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        outputs.append(decoded)
    return outputs
def majority_vote(outputs):
    counts = Counter(outputs)
    return counts.most_common(1)[0][0], counts

In [ ]:
def zero_shot_CoT(text):
  return f"""
  You are a classifier assistant.
  Q: {text['input'] } A: Let's think step by step. Classify the sentiment of this sentence as positive, negative, or neutral:\n{data[i]['input']}. The sentiment is """

In [ ]:
prompt = zero_shot_CoT(data[0])
print(prompt)
samples = self_consistency(prompt, num_samples=5)
best_answer, all_votes = majority_vote(samples)

print("Self-Consistent Answer:", best_answer)
print("All votes:", all_votes)

In [ ]:
def zero_shot_CoT_SC(text, num_samples=5, max_len=50, do_print=False):
        prompt = zero_shot_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        best_answer, all_votes = majority_vote(samples)
        if do_print:
            print('Self-consistent answer: ', best_answer)
            print('All votes: ', all_votes)
        return best_answer, all_votes

In [ ]:
acc = 0
true_results = []
results = []
for i in range(200):
    best_answer, all_votes = zero_shot_CoT_SC(data[i])
    if best_answer.lower() == data[i]['output']:
        acc = acc + 1
        true_results.append(best_answer)
    results.append(best_answer)

In [ ]:
print(acc/200)

In [ ]:
counts = Counter(true_results)
print(counts)
counts = Counter(results)
print(counts)

In [ ]:
acc = 0
true_results = []
results = []
for i in range(200):
    best_answer, all_votes = zero_shot_CoT_SC(data[i],3)
    if best_answer.lower() == data[i]['output']:
        acc = acc + 1
        true_results.append(best_answer.lower())
    results.append(best_answer)
print('Accuracy', acc/200)
counts = Counter(true_results)
print(counts)
counts = Counter(results)
print(counts)

In [ ]:
def select_best_path(thoughts, text, do_print=False):
    prompt = f"""
You are an expert in emotional reasoning. Read the sentence below and analyze the following reasoning paths carefully.

Context: "{text}"

Reasoning paths:
{chr(10).join([
    f"{i+1}. {t}" for i, t in enumerate(thoughts)
])}


Steps:
1. Carefully review the reasoning paths above. Think about emotional polarity, tone, contrast, and subjectivity.
2. Choose the most thoughtful reasoning path (not just a label like 'positive').
3. Output in this exact JSON format:
{{
  "final_sentiment": "positive | negative | neutral",
  "reasoning": "Your explanation using the best reasoning path"
}}

Do not output only the sentiment. Always include explanation inside the JSON block.
"""

    if do_print:
        print('Prompt:\n', prompt)

    input = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
        max_length=350,
        do_sample=False
    )
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()

In [ ]:
    def expand_thoughts( prompt, n=3, max_len=50):
        input = tokenizer(prompt, return_tensors='pt').to('cuda')
        outputs = model.generate(
            **input,
            max_length = max_len,
            num_return_sequences=n,
            do_sample = True,
            top_p=0.95,
            temperature=0.7
        )
        thoughts = [tokenizer.decode(output, skip_special_tokens=True).strip() for output in outputs]
        return thoughts

In [ ]:
def recursive_expand_tree(prompt, depth, breadth, do_print=False):
    if depth == 0:
        return [prompt]

    expanded = expand_thoughts(prompt, n=breadth, max_len=100)
    if do_print:
        print(f"[Depth {depth}] Expand prompt:\n{prompt}")
        print(f"[Depth {depth}] Got thoughts:\n", expanded)

    tree = []
    for thought in expanded:
        sub_tree = recursive_expand_tree(thought, depth - 1, breadth, do_print)
        tree.extend(sub_tree)

    return tree

Zero-shot ToT (basic)

In [ ]:
def zero_shot_ToT( text):
        return f"""
        You are a reasoning classifier assistant. Consider multiple possibilities, classify label for each path (negative, positive, neutral)
         and choose the most reasonable one.
        Q: {text}
        Option 1: Analyze based on emotional keywords.
        Option 2: Consider the context and tone.
        Option 3: Check for contrast or negation.
        Evaluate each option and choose the best.
        Final Answer (label: positive, negative,or neutral ): ?
        """

In [ ]:
acc = 0
true_results = []
results = []
for i in range(200):
    prompt = zero_shot_ToT(data[i]['input'])
    input = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
        max_length=20,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    de = tokenizer.decode(output[0], skip_special_tokens=True)
    if de.lower() == data[i]['output']:
        acc = acc+1
        true_results.append(de)
    results.append(de.lower())
print('Accuracy', acc/200)
counts = Counter(true_results)
print(counts)
counts = Counter(results)
print(counts)

In [ ]:
acc = 0
true_results = []
results = []
for i in range(200):
    prompt = zero_shot_ToT(data[i]['input'])
    input = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
        max_length=200,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    de = tokenizer.decode(output[0], skip_special_tokens=True)
    if de.lower() == data[i]['output']:
        acc = acc+1
        true_results.append(de)
    results.append(de.lower())
print('Accuracy', acc/200)
counts = Counter(true_results)
print(counts)
counts = Counter(results)
print(counts)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("allenai/social_i_qa")

# Hiển thị một mẫu từ tập huấn luyện
print(dataset["train"][0])


In [ ]:
dataset['train']

In [ ]:
def convert_to_instruction_format(example):
    try:
        choices = [example["answerA"], example["answerB"], example["answerC"]]
        label = int(example["label"])
        if label not in [0, 1, 2]:
            return None  # bỏ qua mẫu lỗi
        correct_answer = choices[label]
        return {
            "instruction": "Answer the question (Inference)",
            "input": f"{example['context']} {example['question']} Choices: A) {choices[0]}, B) {choices[1]}, C) {choices[2]}",
            "output": f"Answer: {['A', 'B', 'C'][label]}\nExplanation: {correct_answer}"
        }
    except (KeyError, ValueError, IndexError):
        return None  # bỏ qua nếu có lỗi

# Chuyển đổi toàn bộ tập dữ liệu
converted_data = [convert_to_instruction_format(ex) for ex in dataset['train']]

# Ghi vào tệp JSON
with open('/kaggle/working/reasoning_social.json', 'w', encoding='utf-8') as f:
    json.dump(converted_data, f, ensure_ascii=False, indent=2)

In [ ]:
def wrap_reasoning_to_json(reasoning_text, do_print=False):
    prompt = f"""
Given the reasoning below, output the result in this exact JSON format:
{{
  "final_sentiment": "positive | negative | neutral",
  "reasoning": "your explanation..."
}}

Reasoning:
{reasoning_text}
"""
    if do_print:
        print("[Wrap Prompt]\n", prompt)

    input_ids = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(
        **input_ids,
        max_length=300,
        do_sample=False
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()


In [ ]:
def zero_shot_step_by_step_sentiment(text, do_print=False):
    step_by_step_prompt = f"""
You are a reasoning assistant. Let's analyze the sentiment of the following sentence **step-by-step**.

Sentence: "{text}"

Step 1: Is this sentence factual or emotional?
Step 2: If emotional, describe the emotional tone. If factual, say why it has no emotional tone.
Step 3: Based on the tone and content, what is the most accurate sentiment? (positive, negative, neutral)

Give reasoning clearly and concisely.
"""
    if do_print:
        print("[Prompt]\n", step_by_step_prompt)

    input_ids = tokenizer(step_by_step_prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(
        **input_ids,
        max_length=300,
        do_sample=False
    )
    reasoning = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Wrap into final JSON format
    return wrap_reasoning_to_json(reasoning, do_print=do_print)


Zero-shot ToT BFS

In [ ]:
result = zero_shot_ToT_expanded(data[0]['input'], breadth = 3, do_print=True)
print(result)

**Few-shots**

In [ ]:
def few_shots_direct(text):
        return f"""Classify the sentiment of each sentence as positive, neutral, or negative.
                Sentence: I love this! → positive
                Sentence: This is terrible. → negative
                Sentence: It works as expected. → neutral
                Sentence: {text} →"""


In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
  prompt = few_shots_direct(data[i]['input'])
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de.lower() == data[i]['output']:
    acc = acc+1
    true_results.append(de.lower())
  results.append(de.lower())
print(acc/200)
counts = Counter(results)
print(counts)
counts = Counter(true_results)
print(counts)

Few-shot + CoT (ReAct)

In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
  few_shots_CoT =  f"""
    Classify the sentiment of each sentence as positive, neutral, or negative. Let's explain step by step.
    Sentence: I love this! → Reason: Strong positive words → Answer: positive
    Sentence: This is terrible. → Reason: Strong negative tone → Answer: negative
    Sentence: It works as expected. → Reason: Neutral tone and wording → Answer: neutral
    Sentence: {data[i]['input']} → Reason: Let's think step by step. The sentiment is """
  inputs = tokenizer(few_shots_CoT, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de.lower() == data[i]['output']:
    acc = acc+1
    true_results.append(de.lower())
  results.append(de.lower())
print(acc/200)
counts = Counter(results)
print(counts)
counts = Counter(true_results)
print(counts)

In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
  few_shots_prompt = f"""
  Classify the sentiment of each sentence as positive, neutral, or negative. Let's explain step by step.
                Sentence: I love this! → Reason: Strong positive words → Answer: positive
                Sentence: This is terrible. → Reason: Strong negative tone → Answer: negative
                Sentence: It works as expected. → Reason: Neutral tone and wording → Answer: neutral
                Sentence: {data[i]['input']} → Reason:"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=7,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  results.append(de)
  if de == data[i]['output']:
    acc = acc+1
    true_results.append(de)
print(acc/200)
from collections import Counter
counts = Counter(results)
print('Number of results',counts)
counts = Counter(true_results)
print('Number of true results', counts)

*thử đảo vị trí các ví dụ*

In [ ]:
acc = 0
results = []
true_results = []
for i in range(3, 203):
  few_shots_prompt = f"""Classify the sentiment of each sentence as positive, neutral, or negative. Let's explain step by step.
                Sentence: This is terrible. → Reason: Strong negative tone → Answer: negative
                Sentence: It works as expected. → Reason: Neutral tone and wording → Answer: neutral
                Sentence: I love this! → Reason: Strong positive words → Answer: positive
                Sentence: {data[i]['input']} → Reason:"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de.lower() == data[i]['output']:
    acc = acc+1
    true_results.append(de.lower())
  results.append(de.lower())

In [ ]:
print(acc/200)
print(Counter(true_results))
counts = Counter(results)
print(counts)

In [ ]:
  def few_shots_CoT(text):
    return f"""
You're a sentiment classifier assistant. Classify the sentiment of the sentence into one of the following labels: **negative**, **positive**, or **neutral**. Use chain-of-thought reasoning to explain your answer clearly. Re-evaluate your classification using reasoning, as if double-checking with another LLM.

Examples:

Q: Although I had high hopes, this product was a huge disappointment.
Let's think step by step.
1. The person had high hopes.
2. But the product was a disappointment.
3. The overall sentiment is negative.
A: negative

Q: How can I print mouse coordinates in Rust GTK on a drawing area when clicked?
Let's think step by step.
1. This is a factual and technical question.
2. It expresses no emotion.
3. Therefore, the sentiment is neutral.
A: neutral

Q: I hate this chat
Let's think step by step.
1. The word "hate" expresses strong negative emotion.
2. Therefore, the sentiment is negative.
A: negative

Q: Efficient and fast
Let's think step by step.
1. Both "efficient" and "fast" are positive qualities.
2. Therefore, the sentiment is positive.
A: positive

Q: {text}
Let's think step by step. A: The sentiment is (negative, positive or neutral)?
"""



In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
  inputs = tokenizer(few_shots_CoT(data[i]['input']), return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=7,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  results.append(de.lower())
  if de.lower() == data[i]['output']:
    acc = acc+1
    true_results.append(de.lower())
print('Accuracy ', acc/200)
counts = Counter(true_results)
print(counts)
counts = Counter(results)
print(counts)

Few-shots + CoT + SC

In [ ]:
def few_shots_CoT(text):
        return f"""Classify the sentiment of each sentence as positive, neutral, or negative. Let's explain step by step.
                Sentence: I love this! → Reason: Strong positive words → Answer: positive
                Sentence: This is terrible. → Reason: Strong negative tone → Answer: negative
                Sentence: It works as expected. → Reason: Neutral tone and wording → Answer: neutral
                Sentence: {text} → Reason:"""
def few_shots_CoT_SC(text, num_samples=5, max_len=50, do_print=False):
        prompt = few_shots_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        best_answer, all_votes = majority_vote(samples)
        if do_print:
            print('Answers: ', samples)
            print('Self-consistent answer: ', best_answer)
            print('All votes: ', all_votes)
        return best_answer, all_votes

In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
    best_answer, all_votes = few_shots_CoT_SC(data[i], do_print=True)
    results.append(best_answer.lower())
    if best_answer.lower() == data[i]['output']:
        acc = acc + 1
        true_results.append(best_answer.lower())
print(acc/200)
print(Counter(true_results))
print(Counter(results))

Few-shots + Self-evaluation ToT

In [ ]:
def few_shots_ToT(text):
        return f"""You are a sentiment analysis assistant. For each sentence, explain your reasoning and then classify it as **positive**, **negative**, or **neutral**.

            Example 1:
            Sentence: "The movie was boring but had a good ending."
            Reasoning: Boring → negative, Good ending → positive. The ending left a stronger impression → Sentiment: positive

            Example 2:
            Sentence: "I expected more, but I guess it’s fine."
            Reasoning: Disappointment shows unmet expectations, but “it’s fine” shows acceptance. Mixed feelings → Sentiment: neutral

            Example 3:
            Sentence: "The product is overpriced and doesn’t work well."
            Reasoning: Overpriced and malfunctioning product indicates frustration → Sentiment: negative

            Now analyze the following sentence:
            Sentence: "{text}"
            Reasoning:"""

In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
    prompt = few_shots_ToT(data[i]['input'])
    input = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
        max_length=200,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    de = tokenizer.decode(output[0], skip_special_tokens=True)
    if de.lower() == data[i]['output']:
        acc = acc+1
        true_results.append(de.lower())
    results.append(de.lower())
print('Accuracy', acc/200)
print(Counter(true_results))
print(Counter(results))

In [ ]:
def few_shots_ToT_expanded( text, depth=2, breadth=3, do_print=False):
      root_prompt = f"""You are a sentiment analysis assistant. For each sentence, explain your reasoning and then classify it as **positive**, **negative**, or **neutral**.

            Example 1:
            Sentence: "The movie was boring but had a good ending."
            Reasoning: Boring → negative, Good ending → positive. The ending left a stronger impression → Sentiment: positive

            Example 2:
            Sentence: "I expected more, but I guess it’s fine."
            Reasoning: Disappointment shows unmet expectations, but “it’s fine” shows acceptance. Mixed feelings → Sentiment: neutral

            Example 3:
            Sentence: "The product is overpriced and doesn’t work well."
            Reasoning: Overpriced and malfunctioning product indicates frustration → Sentiment: negative

            Now analyze the following sentence:
            Sentence: "{text}"
            Reasoning:"""
        tree = recursive_expand_tree(root_prompt, depth=depth, breadth=breadth, do_print=do_print)
        best_path = select_best_path(tree, text)
        return best_path

In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
    result = few_shots_ToT_expanded(data[i]['input'], breadth = 3)
    if result.lower() == data[i]['output']:
        acc = acc+1
        true_results.append(result.lower())
    results.append(result)
print(acc/200)
print(Counter(true_results))
print(Counter(results))

In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
    result = few_shots_ToT_expanded(data[i]['input'], depth = 3, breadth = 1)
    if result.lower() == data[i]['output']:
        acc = acc+1
        true_results.append(result.lower())
    results.append(result)
print(acc/200)
print(Counter(true_results))
print(Counter(results))

In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
    result = few_shots_ToT_expanded(data[i]['input'], depth = 2, breadth = 2)
    if result.lower() == data[i]['output']:
        acc = acc+1
        true_results.append(result.lower())
    results.append(result)
print(acc/200)
print(Counter(true_results))
print(Counter(results))

# QA

(hỏi đáp đơn giản)

In [ ]:
clear_gpu()

**Zero-shot**

In [ ]:
import json

with open('/kaggle/input/simple-qa/simple_qa (2).json', 'r', encoding='utf-8') as f:
        qa = json.load(f)

In [ ]:
qa[:3]

In [ ]:
len(qa)

In [ ]:
def zero_shot_direct(text):
        return f"""
        Answer the following question by your knowledge.
        Q: {text}. A: Answer the question
        """

In [ ]:
print(qa[0]['input'])
print(qa[0]['output'])

In [ ]:
input = tokenizer(zero_shot_direct(qa[0]['input']), return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    num_beams=5,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
test_error = """Q: In the early 20th century, where did most of the migrants to the united states come from
. A: Answer the question, which's country ? """

input = tokenizer(test_error, return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))


moi lan mot nuoc ~

In [ ]:
print(qa[1]['input'])
print(qa[1]['output'])

In [ ]:
input = tokenizer(zero_shot_direct(qa[1]['input']), return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Zero-shot + CoT

In [ ]:
def zero_shot_CoT(text):
        return f"""
                Answer the following question by your knowledge.
                Q: {text}. A: Let's think step by step and explain why. Answer the question
                """

In [ ]:
clear_gpu()

In [ ]:
clear_gpu()

In [ ]:
input = tokenizer(zero_shot_CoT(qa[0]['input']), return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    num_beams=5,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

EM

In [ ]:
import re

def extract_final_answer(text):
    match = re.search(r'final answer is\s*(.+?)(?:\.|$)', text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    sentences = re.split(r'[.?!]', text)
    if sentences:
        return sentences[-2].strip()
    return text.strip()


In [ ]:
def exact_match(prediction, ground_truth):
    pred_ans = extract_final_answer(prediction)
    gt_ans = extract_final_answer(ground_truth)

    def normalize_text(text):
        text = text.lower().strip()
        text = re.sub(r'[^\w\s]', '', text)
        return text

    return normalize_text(pred_ans) == normalize_text(gt_ans)

In [ ]:
for i in range(10):
    input = tokenizer(zero_shot_CoT(qa[i]['input']), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)

In [ ]:
from sentence_transformers import SentenceTransformer, util
comp_model = SentenceTransformer('all-MiniLM-L6-V2')

In [ ]:
def cosine_similarity_score(pred, target):
    emb1 = comp_model.encode(pred, convert_to_tensor = True)
    emb2 = comp_model.encode(target, convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2)[0][0])

In [ ]:
print('Target', qa[9]['output'])
print()
print('Prediction', answer)

In [ ]:
cosine_similarity_score(answer, qa[9]['output'])

In [ ]:
cs = []
for i in range(50):
    input = tokenizer(zero_shot_CoT(qa[i]['input']), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)
    c = cosine_similarity_score(answer, qa[i]['output'])
    cs.append(c)

In [ ]:
tem = 0
for i in range(len(cs)):
    tem = tem +cs[i]
print(tem/len(cs))

In [ ]:
def zero_shot_CoT(text):
    return f"""
    Q: {text['input']}. A: Let's think step by step and explain this question clearly, (include definition if it has).
    You can recorrect with other reliable sources such as wikipedia if you need,.."""

In [ ]:
cs = []
for i in range(50):
    input = tokenizer(zero_shot_CoT(qa[i]), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)
    c = cosine_similarity_score(answer, qa[i]['output'])
    cs.append(c)
    print(c)

In [ ]:
tem = 0
for i in range(len(cs)):
    tem = tem +cs[i]
print(tem/len(cs))

Zero-shot + CoT + SC

In [ ]:
cosine = 0
for i in range(3, 53):
    prompt = zero_shot_CoT(qa[i]['input'])
    samples = self_consistency(prompt, num_samples=5)
    for sample in samples:
        print('Sample: ', sample)
        print(cosine_similarity_score(sample, qa[i]['output']))
    print('-------------------------------------------------------')

In [ ]:
for i in range(3, 53):
    prompt = zero_shot_CoT(qa[i]['input'])
    samples = self_consistency(prompt, num_samples=3)
    for sample in samples:
        print('Sample: ', sample)
        print(cosine_similarity_score(sample, qa[i]['output']))
    print('-------------------------------------------------------')

**Few-shots**

In [ ]:
def few_shots_direct(text):
    return f"""
            Answer the following question, using reliable sources such as wikipedia,...
            Q: "between 1900 and 1920 where did most of the migrants to the united states come from",
            A: "During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world."

            Q: "What does inioluwa mean",
            A: "Inioluwa is a Nigerian name that means \"God's gift\" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God."

            Q: "what is CVD",
            A: "CVD stands for Cardiovascular Disease. It is a group of conditions that affect the heart and blood vessels. These conditions include coronary artery disease (which can lead to heart attacks),"

            Q: {text}
            A: Answer the question
                """

In [ ]:
for i in range(3,53):
    input = tokenizer(few_shots_direct(qa[i]['input']), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)
    c = cosine_similarity_score(answer, qa[i]['output'])
    cosine = cosine+c
    print(c)

In [ ]:
print(cosine/50)

Few-shots + CoT

In [ ]:
def few_shots_CoT(text):
        return f"""
        Answer the following question, using reliable sources such as wikipedia,...
        Q: "between 1900 and 1920 where did most of the migrants to the united states come from",
        A: "During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world."

        Q: "What does inioluwa mean",
        A: "Inioluwa is a Nigerian name that means \"God's gift\" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God."

        Q: "what is CVD",
        A: "CVD stands for Cardiovascular Disease. It is a group of conditions that affect the heart and blood vessels. These conditions include coronary artery disease (which can lead to heart attacks),"

        Q: {text}
        A: Let's think step by step and answer the question
                        """

In [ ]:
for i in range(3,53):
    input = tokenizer(few_shots_CoT(qa[i]['input']), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)
    c = cosine_similarity_score(answer, qa[i]['output'])
    cosine = cosine+c
    print(c)

In [ ]:
print(cosine/50)

Few-shots + CoT + SC

In [ ]:
def few_shots_CoT_SC(text, num_samples=5, max_len=50, do_print=False):
        prompt = few_shots_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        best_answer, all_votes = functions.majority_vote(samples)
        if do_print:
            print('Answers: ', samples)
            print('Self-consistent answer: ', best_answer)
            print('All votes: ', all_votes)
        return best_answer, all_votes


In [ ]:
cosine = 0
for i in range(3, 53):
    prompt = few_shots_CoT(qa[i]['input'])
    samples = self_consistency(prompt, num_samples=5)
    for sample in samples:
        print('Sample: ', sample)
        print(cosine_similarity_score(sample, data[i]['output']))
    print('-------------------------------------------------------')

Lấy thư viện task như 1 tập train lớn và test task sẽ là vùng qa còn lại

In [ ]:
# cosine = 0
# def few_shots_CoT_ART(query):
#     examples = find_top_k_tasks(query, 3)
#     prompt = build_prompt(examples, query)
#     return prompt
# for i in range(1000, 1050):
#     input = tokenizer(few_shots_CoT_ART(qa[i]['input']), return_tensors='pt').to('cuda')
#     output = model.generate(
#         **input,
#     max_length = 300,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.95,
#     no_repeat_ngram_size=3,
#     early_stopping=False
#     )
#     answer = tokenizer.decode(output[0], skip_special_tokens=True)
#     print(answer)
#     c = cosine_similarity_score(answer, qa[i]['output'])
#     cosine = cosine+c
#     print(c)

Few-shots + ToT

In [ ]:
# def few_shots_ToT(text):
#     return f"""
#    Your task: Write 3 possible reasoning paths or situations that might lead to different conclusions and response. Do not answer the question
#     {text['input']}
#     """
# def followup_prompt(text):
#  return f"""Here are 3 different reasoning paths:  {text}.
#  You are a reasoning assistant. Your job is to explore different reasoning paths without concluding.
#  Now, expand on each path with clear explanation, but do NOT provide any final answer or conclusion."""


In [ ]:
# for i in range(50):
#     prompt_base = few_shots_ToT(qa[i])
#     strategies = [generate_strategy(prompt_base, j) for j in range(3)]
#     prompt = prompt_base + "\n" + "\n".join(strategies) + "\n" + "Answer:"
#     input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
#     output = model.generate(
#             **input_ids,
#             max_length=200,
#             num_beams=5,
#             no_repeat_ngram_size=2,
#             early_stopping=True
#         )
#     print(qa[i]['input'])
#     answer = tokenizer.decode(output[0], skip_special_tokens=True)
#     print(answer)

# Computation

**Zero-shot**

Zero-shot + CoT

In [ ]:
import json
math = []
with open('/kaggle/input/final-train-math/final_train_math.json', 'r', encoding='utf-8') as f:
    math.append(json.load(f))

In [ ]:
math = math[0]

In [ ]:
prompt = f"Q: {math[0]['input'] } A: Let's think step by step."
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, max_length=160)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Natalia sold 48 / 2 = 24 clips in May. Natalia sold a total of 48 + 24 = 96 clips in April and May. The final answer: 96.


In [ ]:
math[5]['output']

"There are 80/100 * 10 = <<80/100*10=8>>8 more purple flowers than yellow flowers.\nSo in Mark's garden, there are 10 + 8 = <<10+8=18>>18 purple flowers.\nPurple and yellow flowers sum up to 10 + 18 = <<10+18=28>>28 flowers.\nThat means in Mark's garden there are 25/100 * 28 = <<25/100*28=7>>7 green flowers.\nSo in total Mark has 28 + 7 = <<28+7=35>>35 plants in his garden.\n#### 35"

In [ ]:
prompt = f"Q: {math[5]['input'] } A: Let's think step by step. The answer is ?"
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(
    **inputs,
    max_length=100,
    num_beams=2,
    no_repeat_ngram_size=3,
    early_stopping=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

There are 10 * .8 = 8 more purple flowers than yellow flowers. So there are 10 + 8 = 16 purple flowers. And there are 16 * 0.2 = 4 green flowers. Therefore, Mark has 10 + 16 + 4 = 44 flowers in his garden. The answer: 44.


In [ ]:
import re

def extract_answer(text):
    matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', text)
    if matches:
        return matches[-1]
    return None

In [ ]:
answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(answer)

44


In [ ]:
def extract_target(text):
  for line in text.splitlines()[::-1]:
    if line.strip().startswith("####"):
      return line.strip().replace("####","").strip()
  return None

In [ ]:
acc = 0
results = []
for i in range(50):
  zero_shot_prompt = f"""
        You are a math assistant. Solve this question by using math formulas
        Q: {math[i]['input']}. A: Let's think step by step. You need to answer follow this form and always calculate all formulas: The answer is ?
        """

  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )

  answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print('Answer: ',answer)
  target = extract_target(math[i]['output'])
  print('Target: ', target)
  if extract_answer(answer) == target:
    acc = acc+1
  print('----------------------------------------------------------')
print(acc/50)

Answer:  Natalia sold 48 / 2 = 24 clips in May. Altogether, she sold 48 + 24 = 72 clips in April and May. The answer: 72.
Target:  72
----------------------------------------------------------
Answer:  Weng earned $12 x 50 = $600. The answer: 600.
Target:  10
----------------------------------------------------------
Answer:  Betty needs $100 / 2 = $50 more to buy the wallet. Betty's grandparents gave her twice as much as her parents, so they gave her 2 * $15 = $30. So, Betty has $50 + $30 = $70. So Betty still needs to save $100 - $70 = $25 more. The answer: 25.
Target:  5
----------------------------------------------------------
Answer:  Today, Julie was able to read 12 x 2 = 24 pages. She has 120 - 24 = 96 pages left to read. If she wants to read half of the remaining pages tomorrow, she should read 96 / 2 = 54 pages. The answer: 54.
Target:  42
----------------------------------------------------------
Answer:  He writes 3 * 2 = 6 pages a week. So he writes 6 * 52 = 384 pages in a

In [ ]:
acc = 0
results = []
zero_shot_prompt = f"""
 Q: {math[0]['input']}.A: Solve this question by using math formulas. Let's think step by step',
  """
inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=2,
      no_repeat_ngram_size=3,
      early_stopping=True
  )

answer =tokenizer.decode(outputs[0], skip_special_tokens=True)
results.append(answer)

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Natalia sold 48 / 2 = 24 clips in May. Natalia therefore sold a total of 48 + 24 = 72 clips in April and May. The answer: 72.


In [ ]:
print(answer)
print(extract_target(math[1]['output']))

Natalia sold 48 / 2 = 24 clips in May. Natalia therefore sold a total of 48 + 24 = 72 clips in April and May. The answer: 72.
10


In [ ]:
acc = 0
results = []
for i in range(50):
  zero_shot_prompt = f"""
        You are a math assistant. Solve this question by using math formulas
        Q: {math[i]['input']}. A:You need to answer follow this form and always calculate all formulas.: The answer is ?
        """
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=200,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print('Answer: ',answer)
  target = extract_target(math[i]['output'])
  print('Target: ', target)
  if extract_answer(answer) == target:
    acc = acc+1
  print('----------------------------------------------------------')
print(acc/50)

Answer:  48 / 2 = 24 clips were sold in May. Natalia sold a total of 48 + 24 = 96 clips in April and May.
Target:  72
----------------------------------------------------------
Answer:  Weng earned $12 x 50 = $700.
Target:  10
----------------------------------------------------------
Answer:  Betty needs $100 / 2 = $50 more to buy the wallet. Betty's grandparents gave her twice as much as her parents, so they gave her 2 * $15 = $30. Together, her parents and grandparents gave Betty $30 + $15 = $45. Betty still needs $50 - $45 = $25 more.
Target:  5
----------------------------------------------------------
Answer:  Today, Julie was able to read 12 x 2 = 24 pages. So, she has 120 - 24 = 96 pages left to read. If she wants to read half of the remaining pages tomorrow, she should read 96 / 2 = 54 pages.
Target:  42
----------------------------------------------------------
Answer:  He writes 2 * 3 = 6 pages a week. So he writes 6 * 2 = 12 pages per week. That means he wrote 12 * 52 = 552

Zero-shot + CoT + SC

In [ ]:
def self_consistency(prompt, num_samples=5, max_len=150):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
        output = model.generate(
            **inputs,
            max_length=max_len,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        print(decoded)
        outputs.append(decoded)
    return outputs
def majority_vote(outputs):
    answers = [extract_answer(output) for output in outputs]
    counts = Counter(answers)
    return counts.most_common(1)[0][0], counts

In [ ]:
def zero_shot_CoT(text):
        return f"""
        You are a math assistant. Solve this question by using math formulas
        Q: {text}. A: Let's think step by step. You need to answer follow this form and always calculate all formulas: The answer is ?
        """
def zero_shot_CoT_SC(text, num_samples=5, max_len=200, do_print=False):
        prompt = zero_shot_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        best_answer, all_votes = majority_vote(samples)
        if do_print:
            print('All vote: ', all_votes)
            print('Best answer: ', best_answer)
            print('----------------------------------------------')
        return best_answer, all_votes

In [ ]:
from collections import Counter
zero_shot_CoT_SC(math[0]['input'], num_samples = 5, do_print=True)

Natalia sold 48 / 2 = 24 clips in May. Natalia sold 48 + 24 = 72 clips altogether in April and May. The answer: 72.
Natalia sold 48 / 2 = 28 clips in May. Natalia sold 48 + 28 = 88 clips altogether in April and May. The answer: 88.
In May, Natalia sold 48 / 2 = 24 clips. Altogether, Natalia sold 48 + 24 = 72 clips in April and May. The answer: 72.
Natalia sold 48 / 2 = 28 clips in May. Altogether, Natalia sold 48 + 28 = 84 clips in April and May. The answer: 84.
In May, Natalia sold 48 / 2 = 28 clips. Together, Natalia sold 48 + 28 = 88 clips. The answer: 88.
All vote:  Counter({'72': 2, '88': 2, '84': 1})
Best answer:  72
----------------------------------------------


('72', Counter({'72': 2, '88': 2, '84': 1}))

In [ ]:
acc = 0
for i in range(50):
    best_answer, all_votes = zero_shot_CoT_SC(math[i]['input'], do_print=True)
    if best_answer == extract_target(math[i]['output']):
        acc = acc+1

Natalia sold 48 / 2 = 24 clips in May. Therefore, Natalia sold a total of 48 + 24 = 96 clips in April and May. The answer: 96.
Natalia sold 48 / 2 = 28 clips in May. Natalia sold 48 + 28 = 80 clips altogether in April and May. The answer: 80.
Natalia sold 48 / 2 = 28 clips in May. Altogether, she sold 48 + 28 = 76 clips in April and May. The answer: 76.
Natalia sold 48 / 2 = 24 clips in May. Natalia sold a total of 48 + 24 = 96 clips in April and May. The answer: 96.
Natalia sold 48 / 2 = 24 clips in May. Natalia sold a total of 48 + 24 = 96 clips in April and May. The answer: 96.
All vote:  Counter({'96': 3, '80': 1, '76': 1})
Best answer:  96
----------------------------------------------
Weng earned $12 x 50 = $180. The answer: 180.
Weng earns $12 x 50 = $120. The answer: 120.
Weng earned $12 x 50 = $600. The answer: 600.
Weng earns $12 x 50 = $700 for babysitting yesterday. The answer: 700.
Weng earned $12 x 50 = $600. The answer: 600.
All vote:  Counter({'600': 2, '180': 1, '120':

In [ ]:
print(acc/50)

0.1


Zero-shot + ToT

In [10]:
def generate_output(type, input, max_len=200):
        if type=='generation':
            output = model.generate(
                **input,
                max_length = max_len,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                num_return_sequences=1
            )
        else:
            output = model.generate(
                **input,
                max_length = max_len,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True
            )
        return output

In [ ]:
def zero_shot_ToT(text):
        return f"""
        You are a math assistant. Solve this question by identifying multiple thoughts or approaches.
        Q: {text} A: Let's try solving this from different reasoning paths and always calculate all formulas. The answer is ?
        """
acc = 0
for i in range(50):
    prompt = zero_shot_ToT(math[i]['input'])
    input = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(
                **input,
                max_length = 200,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True
            )
    answer = tokenizer.decode(output[0],
                      skip_special_tokens=True)
    print('Output: ', answer)
    print('Answer: ', extract_answer(answer))
    print('Target: ', extract_target(math[i]['output']))
    if extract_answer(answer)== extract_target(math[i]['output']):
        acc = acc + 1

Output:  First find the total number of clips Natalia sold in April: 48 clips * 2 = 96 clips. Then find how many clips she sold half as many in May: 72 clips / (2 * 48) = 24 clip. Now add the two amounts to find 48 + 24 = 72. Finally, subtract 72 from both sides of the equation to get 48 - 72 = 12 clips sold altogether. Thus, the answer is 12.
Answer:  12
Target:  72
Output:  Weng earned $12 x 50 = $600. Thus, the answer is 600.
Answer:  600
Target:  10
Output:  Betty needs $100 / 2 = $50 more to buy the wallet. Her grandparents gave her twice as much as her parents, which means they give her 2 * $15 = $30. So, the total amount of money she received is $15 + $30 = $55. Therefore, she still needs $50 - $55 = $25 more. The answer: 25.
Answer:  25
Target:  5
Output:  Today, Julie was able to read 12 x 2 = 24 pages. The total number of pages that Julie has read is 12 + 24 = 36. There are 120 - 36 = 72 pages left in the book. Tomorrow, she should read 1 / 2 of the remaining pages, i.e., 72(

In [ ]:
print(acc/50)

0.04


Kết quả rất tệ nếu không hề có 1 ví dụ nào cho mô hình học

**Few-shots**

Few-shots + CoT

In [ ]:
len(math)

7473

In [ ]:
acc  = 0
results = []
for i in range(3, 53):
  few_shots_prompt = f"""
  You are a math assistant. Whenever you see a math expression, call the function CALC().
Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
A: Natalia sold CALC(48/2)=24 clips in May.
Natalia sold CALC(48+24)=72 clips altogether in April and May.
#### 72
Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
A: Weng earns $CALC(12/60)=0.2 per minute.
Working 50 minutes, she earned $CALC(0.2*50)=10,
#### 10
Q: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
A: In the beginning, Betty has only $CALC(100/2) = 50.
Betty's grandparents gave her $CALC(15*2)=30.
This means, Betty needs $CALC(100-50-30-15) = 5 more.
#### 5
Q: {math[i]['input'] } A: Let's think step by step.  Solve this question by using math formulas. The answer is ?"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )

  answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
  print('Output: ', tokenizer.decode(outputs[0], skip_special_tokens=True))
  print('Answer: ', answer)
  print('Target: ', extract_target(math[i]['output']))
  print('-----------------------------------------------------------------------------------------')
  target = extract_target(math[i]['output'])
  if answer == target:
    acc = acc+1


Output:  Today, Julie was able to read 12 x 2 = 24 pages. So, she is left with 120 - 24 = 96 pages. If she wants to read half of the remaining pages tomorrow, she should read 96 / 2 = 54 pages. #### 54
Answer:  54
Target:  42
-----------------------------------------------------------------------------------------
Output:  He writes 2 * 3 = 6 pages a week. So he writes 6 * 2 = 12 pages per week. That means he wrote 12 * 52 = 552 pages in a year. #### 552
Answer:  552
Target:  624
-----------------------------------------------------------------------------------------
Output:  There are 10 * .8 = 8 more purple flowers than yellow flowers in Mark's garden. So there are 10 + 8 = 16 purple flowers in his garden. And there are 16 * 0.25 = 4 green flowers. So in total, Mark has 10 + 16 + 4 = 44 flowers. #### 44
Answer:  44
Target:  35
-----------------------------------------------------------------------------------------
Output:  He eats 2 large pizzas * 16 slices / large pizza = 32 slice

In [ ]:
print(acc/50)

0.1


In [ ]:
def few_shots_CoT(text):
        return f"""Solve this math problem step by step:
Q: A store offers a 30% discount on a $200 item. What's the price after discount?
Step 1: Calculate 30% of 200 = 0.3 * 200 = 60
Step 2: Subtract from 200 → 200 - 60 = 140
A: $140

Now solve:
{text}
Step-by-step reasoning:
"""
inputs = tokenizer(few_shots_CoT(math[3]['input']), return_tensors="pt").to('cuda')
outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )
answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
print('Output: ', tokenizer.decode(outputs[0], skip_special_tokens=True))
print('Answer: ', answer)
print('Target: ', extract_target(math[3]['output']))
print('-----------------------------------------------------------------------------------------')

Output:  Today, Julie was able to read 12 x 2 = 24 pages. Thus, she has 120 - 12 - 24 = 72 pages left to read. If she wants to read half of the remaining pages tomorrow, she should read 72 / 2 = 36 pages. So the answer is 36.
Answer:  36
Target:  42
-----------------------------------------------------------------------------------------


In [ ]:
clear_gpu()

In [ ]:
def few_shots_CoT(text):
        return f"""
        You are a math assistant. Whenever you see a math expression, call the function CALC().
        Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
        A: Natalia sold CALC(48/2)=24 clips in May.
        Natalia sold CALC(48+24)=72 clips altogether in April and May.
        #### 72
        Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
        A: Weng earns $CALC(12/60)=0.2 per minute.
        Working 50 minutes, she earned $CALC(0.2*50)=10,
        #### 10
        Q: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
        A: In the beginning, Betty has only $CALC(100/2) = 50.
        Betty's grandparents gave her $CALC(15*2)=30.
        This means, Betty needs $CALC(100-50-30-15) = 5 more.
        #### 5
        Q: {text} A: Let's think step by step and always calculate all formulas. The answer is ?
        """

In [ ]:
acc  = 0
results = []
for i in range(3, 53):
  inputs = tokenizer(few_shots_CoT(math[i]['input']), return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )
  target = extract_target(math[i]['output'])
  answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
  print('Output: ', tokenizer.decode(outputs[0], skip_special_tokens=True))
  print('Answer: ', answer)
  print('Target: ', target)
  print('-----------------------------------------------------------------------------------------')
  if answer == target:
    acc = acc+1


Output:  Today, Julie was able to read 12 x 2 = 24 pages. So, she is left with 120 - 24 = 72 pages. If she wants to read half of the remaining pages tomorrow, she should read 72 / 2 = 36 pages. #### 36
Answer:  36
Target:  42
-----------------------------------------------------------------------------------------
Output:  He writes 2 * 3 = 6 pages a week. So he writes 6 * 2 = 12 pages per week. He writes 12 * 52 = 552 pages per year. #### 552
Answer:  552
Target:  624
-----------------------------------------------------------------------------------------
Output:  There are 10 * .8 = 8 more purple flowers than yellow flowers in Mark's garden. So there are 10 + 8 = 16 purple flowers in his garden. And there are 16 * 0.25 = 4 green flowers. So in total, Mark has 10 + 16 + 4 = 44 flowers. #### 44
Answer:  44
Target:  35
-----------------------------------------------------------------------------------------
Output:  He eats 2 * 16 + 2 * 8 = 32 pieces of pizza. #### 32
Answer:  32
Targe

In [ ]:
print(acc/50)

0.08


In [ ]:
def split_cot_and_answer(output_text):
    lines = output_text.strip().splitlines()
    cot_lines = [line for line in lines if not line.strip().startswith("####")]
    answer = extract_target(output_text)
    return "\n".join(cot_lines), answer

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
task_samples = math[:500]
new_tasks = []
for task in task_samples:
    cot, answer = split_cot_and_answer(task["output"])
    task["chain_of_thought"] = cot
    task["answer"] = answer
    new_tasks.append(task)
texts = [task['input'] for task in new_tasks]
vectorizer = TfidfVectorizer().fit(texts)
X = vectorizer.transform(texts)

In [ ]:
def find_top_k_tasks(text, k=3):
        q = vectorizer.transform([text])
        scores = cosine_similarity(q, X).flatten()
        top = scores.argsort()[::-1][:k]
        return [new_tasks[i] for i in top]

In [ ]:
def find_similar_task(query):
    q = vectorizer.transform([query])
    idx = cosine_similarity(q, X).argmax()
    return new_tasks[idx]

In [ ]:
test_tasks = math[1000:]

Few-shots + CoT + ART

In [ ]:
def build_prompt(examples, query):
    prompt = ""
    for ex in examples:
        cot = "\n".join(ex.get("chain_of_thought", [])) if "chain_of_thought" in ex else ex["output"]
        prompt += f"""Task: {ex.get('instruction', 'Math Problem')}
                    Input: {ex['input']}
                    Chain of Thought:
                    {cot}
                    Output: {ex['output']}

                    """

    prompt += (
        "Now solve this task:\n"
        f"Input: {query}\n"
        "Solution:\n"
        "Let's solve this step-by-step. Write your reasoning clearly. Final answer: ####"
    )
    return prompt

In [ ]:
def few_shots_CoT_ART(query, k = 3):
    examples = find_top_k_tasks(query, k)
    prompt = build_prompt(examples, query)
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(
      **inputs,
      max_length=150,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [31]:
clear_gpu()

In [32]:
acc = 0
for i in range(50):
    answer = few_shots_CoT_ART(test_tasks[i]['input'])
    print('Output: ', answer)
    print('Answer: ', extract_answer(answer))
    print('Target: ', extract_target(test_tasks[i]['output']))

Output:  
          Task: Reasoning for text
          Input: Information:

- Jan Karol Chodkiewicz ("c". 1560  24 September 1621 ) was a military commander of the PolishLithuanian Commonwealth army who was from 1601 Field Hetman of Lithuania, and from 1605 Grand Hetman of Lithuania, and was one of the most prominent noblemen and military commanders of the PolishLithuanian Commonwealth of his era. His coat of arms was Chodkiewicz, as was his family name.

- Chernivtsi Oblast ("erniveka oblas") is an oblast (province) in western Ukraine, consisting mostly of the northern part of the region of Bukovina, and bordering Romania and Moldova. It has a large variety of landforms: the Carpathian Mountains and picturesque hills at the foot of the mountains gradually change to a broad partly forested plain situated between the Dniester and Prut rivers. Its capital is the city Chernivtsi. The region has a population of 913,275 and spans 8,100 km².

- The Battle of Khotyn or Battle of Chocim or Hot

TypeError: extract_answer() missing 1 required positional argument: 'answer'

In [ ]:
clear_gpu()

In [ ]:
acc = 0
for i in range(50):
    answer = few_shots_CoT_ART(test_tasks[i]['input'], 5)
    print('Output: ', answer)
    print('Answer: ', extract_answer(answer))
    print('Target: ', extract_target(test_tasks[i]['output']))

Làm sao để giải quyết được OOM: prompt dài ??? -> prompt compression

Few-shots + CoT(ReAct) + SELF-CONSITENCY (SC) + ART

In [ ]:
def self_consistency(prompt, num_samples=5):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
        output = model.generate(
            **inputs,
            max_length=150,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        print(decoded)
        outputs.append(decoded)
    return outputs
def majority_vote(outputs):
    answers = [extract_answer(out) for out in outputs]
    counts = Counter(answers)
    return counts.most_common(1)[0][0], counts

In [ ]:
from collections import Counter
def few_shots_CoT_SC(query):
    prompt = few_shots_CoT(query)
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)

    print("Self-Consistent Answer:", best_answer)
    print("All votes:", all_votes)
    return best_answer

In [ ]:
acc= 0
for i in range(50):
    if few_shots_CoT_SC(math[i]['input']) == extract_target(math[i]['output']):
        acc = acc + 1
    print('Target: ', extract_target(math[i]['output']))
    print('--------------------------------------------------------------')

Natalia sold CALC(48/2) = 24 clips in May. Natalia sold CALC(48+24) = 72 clips altogether in April and May. #### 72
Natalia sold CALC(48/2)=24 clips in May. Natalia sold CALC(48+24)=72 clips altogether in April and May. #### 72
Natalia sold CALC(48/2)=24 clips in May. Natalia sold CALC(48+24)=72 clips altogether in April and May. #### 72
Natalia sold CALC(48/2)=24 clips in May. Natalia sold CALC(48+24)=72 clips altogether in April and May. #### 72
Natalia sold 48 / 2 = 24 clips in May. Natalia sold 48 + 24 = 72 clips altogether in April and May. #### 72
Self-Consistent Answer: 72
All votes: Counter({'72': 5})
Target:  72
--------------------------------------------------------------
Weng earns $CALC(12/60)=0.2 per minute. Working 50 minutes, she earned $CALC(0.2*50)=10, #### 10
Weng earns $CALC(12/60) = 0.2 per minute. Working 50 minutes, she earned $CALC(0.2*50)=10, #### 10
Weng earns $CALC(12/60)=0.2 per minute. Working 50 minutes, she earned $CALC(0.2*50)=10, #### 10
Weng earns $CAL

In [ ]:
def build_prompt(examples, query):
    prompt = ""
    for ex in examples:
        cot = "\n".join(ex.get("chain_of_thought", [])) if "chain_of_thought" in ex else ex["output"]
        prompt += f"""Task: {ex.get('instruction', 'Math Problem')}
                    Input: {ex['input']}
                    Chain of Thought:
                    {cot}
                    Output: {ex['output']}

                    """

    prompt += (
        "Now solve this task:\n"
        f"Input: {query}\n"
        "Solution:\n"
        "Let's solve this step-by-step. Write your reasoning clearly. Final answer: ####"
    )
    return prompt


In [ ]:
def few_shots_CoT_SC_ART(query):
    examples = find_top_k_tasks(query, 3)
    prompt = build_prompt(examples, query)
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)

    print("Self-Consistent Answer:", best_answer)
    print("All votes:", all_votes)

In [ ]:
for i in range(20):
    few_shots_CoT_SC(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

Few-shots + ToT

In [ ]:
    def few_shots_ToT(text):
        return f"""
        You are a math assistant.\n
         Use a tree-of-thought approach to break down complex problems by exploring different solution paths, then converge on the correct answer.

        Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
        A:
        Thought 1: Natalia sold 48 clips in April.
        Thought 2: She sold half as many in May → CALC(48/2)=24 clips.
        Thought 3: Total clips = CALC(48+24)=72.
        The final answer is 72

        Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
        A:
        Thought 1: Convert hourly rate to per-minute → CALC(12/60)=0.2.
        Thought 2: Multiply by minutes worked → CALC(0.2*50)=10.
        The final answer is 10

        Q: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
        A:
        Thought 1: Betty initially has CALC(100/2)=50.
        Thought 2: Parents give her $15, grandparents give CALC(15*2)=30.
        Thought 3: Total amount Betty has = CALC(50+15+30)=95.
        Thought 4: She still needs CALC(100-95)=5 more.
        The final answer is 5

        Q: {text}
        A:
        Let's think in steps, exploring each possibility:
        Thought 1:
        Thought 2:
        Thought 3:
        Always calculate all formulas.The answer is?
        """


In [ ]:
clear_gpu()

In [ ]:
for i in range(3, 53):
    prompt = few_shots_ToT(math[i]['input'])
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
            **input_ids,
            max_length=200,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    target = extract_target(math[i]['output'])
    if extract_answer(answer) == target:
        acc = acc + 1
    print('Output: ', answer)
    print('Answer: ', extract_answer(answer))
    print('Target: ', target)
    print('-------------------------------------------------------------------------------------')

Output:  Thought 1: Today, Julie was able to read 12 x 2 = 24 pages. So, she has 120 - 24 = 96 pages left in the book. If she reads half of the remaining pages tomorrow, then she should read (96 / 2) * .5 = 36 pages more. The final answer is 36
Answer:  36
Target:  42
-------------------------------------------------------------------------------------
Output:  Thought 1: He writes 3 * 2 = 6 pages a week. So he writes 6 * 52 = 384 pages per year. The final answer is 394.
Answer:  384
Target:  624
-------------------------------------------------------------------------------------
Output:  Thought 1: There are 10 * .8 = 8 more purple flowers than yellow flowers in Mark's garden. So he has 10 + 8 = 16 purple plants. And there are 16 * 0.25 = 4 green plants in his garden as well. Thus, Mark has 16 + 4 = 24 flowers. The final answer is 24
Answer:  24
Target:  35
-------------------------------------------------------------------------------------
Output:  Thought 1: He eats 2 * 16 = 32 sl

In [ ]:
    def expand_thoughts(prompt, n=3, max_len=100):
        input = tokenizer(prompt, return_tensors='pt').to('cuda')
        outputs = model.generate(
            **input,
            max_length = max_len,
            num_return_sequences=n,
            do_sample = True,
            top_p=0.95,
            temperature=0.7
        )
        thoughts = [tokenizer.decode(output, skip_special_tokens=True).strip() for output in outputs]
        return thoughts

In [ ]:
import re

def get_best_thought(thoughts, text, do_print=False):
    output = select_best_path(thoughts, text, do_print=do_print)
    if len(output) < 3:
        match = re.search(r"\b(\d+)\b", output)
        index = int(match.group(1)) - 1
        if 0 <= index < len(thoughts):
            return thoughts[index]
    else:
        return output

In [ ]:
def zero_shot_ToT(text):
    return f"""You are a math assistant. For the question below, brainstorm multiple reasoning paths before concluding the answer.

Q: {text}
Thought 1:"""
def few_shots_ToT(text):
        return f"""
        You are a math assistant.\n
         Use a tree-of-thought approach to break down complex problems by exploring different solution paths, then converge on the correct answer.

        Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
        A:
        Thought 1: Natalia sold 48 clips in April.
        Thought 2: She sold half as many in May → CALC(48/2)=24 clips.
        Thought 3: Total clips = CALC(48+24)=72.

        Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
        A:
        Thought 1: Convert hourly rate to per-minute → CALC(12/60)=0.2.
        Thought 2: Multiply by minutes worked → CALC(0.2*50)=10.

        Q: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
        A:
        Thought 1: Betty initially has CALC(100/2)=50.
        Thought 2: Parents give her $15, grandparents give CALC(15*2)=30.
        Thought 3: Total amount Betty has = CALC(50+15+30)=95.
        Thought 4: She still needs CALC(100-95)=5 more.

        Q: {text}
        A:
        Let's think in steps, exploring each possibility:
        Thought 1:
        Thought 2:
        Thought 3:
        """


def expand_thoughts(prompt, n=3):
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = [model.generate(**inputs, max_length=150) for _ in range(n)]
    return [tokenizer.decode(o[0], skip_special_tokens=True) for o in outputs]

def recursive_expand_tree(prompt, depth, breadth, context, do_print=False):
    if depth == 0:
        return [prompt]

    # Gắn lại context đề bài vào mỗi bước mở rộng
    base_prompt = f"""
        You are a math assistant. Use tree-of-thought reasoning to explore solutions.

        Question: {context}

        Partial Reasoning:
        {prompt}
        Using math formulas
        Now expand with next steps:
        """

    expanded = expand_thoughts(base_prompt, n=breadth)
    expanded = list(set(expanded))
    if do_print:
        print(f"[Depth {depth}] Base prompt:\n{base_prompt}")
        print(f"[Depth {depth}] Got thoughts:\n", expanded)

    tree = []
    for thought in expanded:
        sub_tree = recursive_expand_tree(thought, depth - 1, breadth, context, do_print)
        tree.extend(sub_tree)
    return tree


def select_best_path(thoughts, text, do_print=False):
    prompt = f"""You are a math expert. Given a question and several reasoning paths, choose the most logical one.

    Question: {text}

    Candidates:
    {chr(10).join([f"{i + 1}. {t}" for i, t in enumerate(thoughts)])}

    Reply with the best option, justify briefly. Final answer is ?"""
    if do_print:
            print('prompt:\n', prompt)
    print(thoughts)
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(**inputs, max_length=300)
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()

def zero_shot_ToT_expanded(text, depth=2, breadth=3, do_print=False):
    root_prompt = zero_shot_ToT(text)
    if do_print:
            print('Root_prompt:\n', root_prompt)
    tree = recursive_expand_tree(root_prompt, depth=depth, breadth=breadth, do_print=do_print)
    best = select_best_path(tree, text, do_print)
    return best
def few_shots_ToT_expanded(text, depth=2, breadth=3, do_print=False):
    root_prompt = few_shots_ToT(text)

    if do_print:
        print('[Root prompt]')
        print(root_prompt)

    tree = []
    for _ in range(breadth):
        single_tree = recursive_expand_tree(root_prompt, depth, breadth, text, do_print)
        tree.extend(single_tree)

    best_path = get_best_thought(tree, text, do_print)
    return best_path


In [ ]:
    context = math[0]['input']
    answer = few_shots_ToT_expanded(math[0]['input'], depth = 2, breadth =2, do_print=True)
    target = extract_target(math[0]['output'])
    print('Output: ', answer)
    print('Target: ', target)
    print('-------------------------------------------------------------------------------------')

[Root prompt]

        You are a math assistant.

         Use a tree-of-thought approach to break down complex problems by exploring different solution paths, then converge on the correct answer.

        Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
        A:
        Thought 1: Natalia sold 48 clips in April.
        Thought 2: She sold half as many in May → CALC(48/2)=24 clips.
        Thought 3: Total clips = CALC(48+24)=72.

        Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
        A:
        Thought 1: Convert hourly rate to per-minute → CALC(12/60)=0.2.
        Thought 2: Multiply by minutes worked → CALC(0.2*50)=10.

        Q: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her gran

In [ ]:
for i in range(3, 53):
    answer = few_shots_ToT_expanded(math[i]['input'], depth = 2, breadth =2, do_print=False)
    target = extract_target(math[i]['output'])
    print('Output: ', answer)
    print('Answer: ', extract_answer(answer))
    print('Target: ', target)
    print('-------------------------------------------------------------------------------------')

['Yesterday, Julie was able to read 12 pages. Today, she was able to read 12 * 2 = 24 pages. So, she has 120 - 24 = 96 pages left to read. If she wants to read half of the remaining pages tomorrow, she should read 96 / 2 = 54 pages. Thus, the answer is 54.', 'Yesterday, Julie was able to read 12 pages. Today, she was able to read 12 * 2 = 24 pages. So, she has 120 - 24 = 96 pages left to read. If she wants to read half of the remaining pages tomorrow, she should read 96 / 2 = 54 pages. Thus, the answer is 54.']
Output:  Yesterday, Julie was able to read 12 pages. Today, she was able to read 12 * 2 = 24 pages. So, she has 120 - 24 = 96 pages left to read. If she wants to read half of the remaining pages tomorrow, she should read 96 / 2 = 54 pages. Thus, the answer is 54.
Answer:  54
Target:  42
-------------------------------------------------------------------------------------
['He writes 3 * 2 = 6 pages a week. So he writes 6 * 2 = 12 pages a week. So he writes 12 * 52 = 552 pages a 

In [ ]:
acc = 0
for i in range(3, 53):
    answer = few_shots_ToT_expanded(math[i]['input'], depth = 3, breadth =2, do_print=False)
    target = extract_target(math[i]['output'])
    if extract_answer(answer) == target:
        acc = acc + 1
    print('Output: ', answer)
    print('Answer: ', extract_answer(answer))
    print('Target: ', target)
    print('-------------------------------------------------------------------------------------')

NameError: name 'few_shots_ToT_expanded' is not defined

# Reasoning

In [24]:
import json
data= []
file_path = "/content/reasoning (1).json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

In [25]:
len(data)

403

In [ ]:
data[1]

{'source': "In this task, you are provided with an article of the legal acts. Your task is to classify it into three categories (Regulation, Decision and Directive) based on its content: 1) Regulation is a binding legislative act that must be applied in its entirety on a set date across all the member states (European Union countries). 2) Decision is binding on those to whom it is addressed (e.g. an European Union country or an individual company) and is directly applicable. 3) Directive is a legislative act that sets out a goal that all  must achieve. However, it is up to the individual countries to devise their own laws on how to reach these goals.\n\nProof that the special export tax mentioned in Articles 2 and 3 of Regulation (EEC) No 1234/71 has been paid shall be furnished to the competent authority of the importing Member State by presentation of movement certificate A.TR.1. In that case, one of the following entries shall be made in the 'Remarks' section by the competent author

**Zero-shot**

In [5]:
from sentence_transformers import SentenceTransformer, util
comp_model = SentenceTransformer('all-MiniLM-L6-V2')
def cosine_similarity_score(pred, target):
    emb1 = comp_model.encode(pred, convert_to_tensor = True)
    emb2 = comp_model.encode(target, convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2)[0][0])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def zero_shot_direct(text):
        return f"""
        You are a reasoning assistant. Break down the problem by exploring multiple lines of reasoning.
        Q: {text}. A:Answer the question and explain clearly. Answer: ....
        """

In [ ]:
def generate_output(type, input, max_len=400):
        if type=='generation':
            output = model.generate(
                **input,
                max_length = max_len,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                num_return_sequences=1
            )
        else:
            output = model.generate(
                **input,
                max_length = max_len,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True
            )
        return output

In [ ]:
prompt = zero_shot_direct(data[0]['input'])
input = tokenizer(prompt, return_tensors='pt').to('cuda')
answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
print(answer)

The relevant information is: Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants


In [ ]:
print(data[0]['output'])
print(data[0]['explanation'])

Answer: from plants
The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of the medicines found in plants mentioned by the author. Thus it can be stated with certainty that some medicines do indeed come from plants.

Therefore, "from plants" is the correct answer option to this question based on the context provided."


In [7]:
import re

def extract_answer(text, answer):
    pattern = re.escape(answer)
    match = re.search(pattern, text)
    if match:
        return True
    else:
        return False

def extract_target(text):
    match = re.search(r'Answer:\s*(.*)', text)
    if match:
        return match.group(1).strip()
    else:
        return None


In [ ]:
extract_answer(answer, extract_target(data[0]['output']))

True

In [ ]:
data[9]['output']

'Answer: Where to watch German world cup soccer games in Lisbon?'

In [ ]:
acc=0
for i in range(50):
  prompt = zero_shot_direct(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  if extract_answer(answer, extract_target(data[i]['output'])):
    acc = acc+1
  print(answer)
  print(data[i]['output'])


The relevant information is: Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants
Answer: from plants
Regulation
Answer: Regulation
The relevant information to answer the above question is: Robert E. Lee surrendered to Ulysses S. Grant at Appomattox Court House, Virginia, on June 9, 1865, ending the American Civil War. Therefore, the final answer is .
Answer: Battle of Appomattox Court House
major heart attacks
Answer: major heart attacks
Red Skelton
Answer: Red Skelton
It cannot be inferred that the Normandy landings took place in June 1944. Therefore, the final answer is it is not possible to tell.
Answer: no
Sue has sickle cell trait, which causes her hemoglobin molecules to aggregate. Therefore, the final answer is Sue.
Answer: Sue
The lips. Therefore, the final answer is the lips
Answer: lips
No one stroked Mama's hair and let the child weep. Therefore, the answer is no.
Answe

In [ ]:
print(acc/50) #khớp hoàn toàn

0.64


nhiều output ngoại lệ nên cần kiểm tra EM or AEM thủ công

In [ ]:
cosine = 0
for i in range(50):
  prompt = zero_shot_direct(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, data[i]['explanation'])
  cosine = cosine + c
  print(c)
  print(answer)
  print(data[i]['explanation'])


0.8729904294013977
The relevant information is: Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants
The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of the medicines found in plants mentioned by the author. Thus it can be stated with certainty that some medicines do indeed come from plants.

Therefore, "from plants" is the correct answer option to this question based on the context provided."
0.5346497893333435
Regulation
This act has all the features of Regulation. The act is addressed to an European Union country; it must be applied once and for all within a certain time-limit; it lays down general rules of application, which are binding on all Member States.
0.7471153736114502
The relevant information to answer the above 

In [ ]:
print(cosine/50)

0.4962591398693621


In [ ]:
def zero_shot_direct(text):
        return f"""
        You are a reasoning assistant.
        Q: {text}. A:Answer the question and explain clearly. Answer: .... Explantion:....
        """

In [ ]:
acc=0
for i in range(50):
  prompt = zero_shot_direct(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  if extract_answer(answer, extract_target(data[i]['output'])):
    acc = acc+1
  print(answer)
  print(data[i]['output'])


plants
Answer: from plants
Regulation
Answer: Regulation
Lee's Raid
Answer: Battle of Appomattox Court House
major heart attacks
Answer: major heart attacks
Red Skelton
Answer: Red Skelton
it is not possible to tell
Answer: no
Sue has sickle cell trait, which causes her hemoglobin molecules to aggregate. So the final answer is Sue.
Answer: Sue
lips
Answer: lips
No one stroked Mama's hair and let the child weep.
Answer: no
Where to watch the Germany vs Sweden World Cup match in Lisbon?
Answer: Where to watch German world cup soccer games in Lisbon?
Yes
Answer: Yes
E.
Answer: E
0
Answer: 0
Yes
Answer: Yes
a).
Answer: Great I just got it! Says it will be here in 35 minutes. Thank you so much! You have a great day!
B
Answer: B
Hieronymus
Answer: Hieronymus
Ben
Answer: Ben
Begin work is the action of starting something. Hunger is an effective motivator for begin work. The final answer: begin w.
Answer: begin work
Harold
Answer: Harold
How do gymnasts alter the shape of the landing area?
Ans

In [ ]:
print(acc/50)

0.6


In [ ]:
cosine = 0
for i in range(50):
  prompt = zero_shot_direct(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, data[i]['explanation'])
  cosine = cosine + c
  print(c)
  print(answer)
  print(data[i]['explanation'])


0.5209393501281738
plants
The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of the medicines found in plants mentioned by the author. Thus it can be stated with certainty that some medicines do indeed come from plants.

Therefore, "from plants" is the correct answer option to this question based on the context provided."
0.5346497893333435
Regulation
This act has all the features of Regulation. The act is addressed to an European Union country; it must be applied once and for all within a certain time-limit; it lays down general rules of application, which are binding on all Member States.
0.28436779975891113
Lee's Raid
The Battle of Appomattox Court House was a battle in the final stages of the American Civil War, resulting in Confederate General Robert E. Lee surrendering his Army to Union Commander Ulysses S. Grant on April 9th

In [ ]:
print(cosine/50)

0.4113481701351702


Zero-shot + CoT

In [ ]:
def zero_shot_CoT(text):
    return f"Q: {text}. A: Let's think step by step and explain"

In [ ]:
acc=0
for i in range(50):
  prompt = zero_shot_CoT(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  if extract_answer(answer, extract_target(data[i]['output'])):
    acc = acc+1
  print(answer)
  print(data[i]['output'])


The important part for answering this question is: Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants
Answer: from plants
Regulation
Answer: Regulation
Lee's Island
Answer: Battle of Appomattox Court House
major heart attacks
Answer: major heart attacks
Red Skelton
Answer: Red Skelton
It's impossible to say when the Normandy landings took place. So, the answer is no.
Answer: no
Sue. So, the answer is .
Answer: Sue
The important part for answering this question is: Red hair varies from a deep burgundy through burnt orange to bright copper. It is characterized by high levels of the reddish pigment pheomelanin
Answer: lips
No one stroked Mama's hair and let the child weep. So, the answer is no.
Answer: no
Where to watch Germany vs. Sweden in Lisbon?
Answer: Where to watch German world cup soccer games in Lisbon?
Yes
Answer: Yes
E.
Answer: E
0
Answer: 0
Yes
Answer: Yes
The correct an

In [ ]:
print(acc/50)

0.66


In [ ]:
cosine = 0
for i in range(50):
  prompt = zero_shot_CoT(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, data[i]['explanation'])
  cosine = cosine + c
  print(c)
  print(answer)
  print(data[i]['explanation'])


0.8767884373664856
The important part for answering this question is: Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants
The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of the medicines found in plants mentioned by the author. Thus it can be stated with certainty that some medicines do indeed come from plants.

Therefore, "from plants" is the correct answer option to this question based on the context provided."
0.5346497893333435
Regulation
This act has all the features of Regulation. The act is addressed to an European Union country; it must be applied once and for all within a certain time-limit; it lays down general rules of application, which are binding on all Member States.
0.2584016025066376
Lee's Island
The Battle

In [ ]:
print(cosine/50)

0.4527788890339434


In [ ]:
clear_gpu()

In [ ]:
def zero_shot_CoT(text):
    return f"""
    You are a reasoning assistant. Break down the problem by exploring multiple lines of reasoning.
    Q: {text}. A: Let's think step by step and explain"""

In [ ]:
acc=0
for i in range(50):
  prompt = zero_shot_CoT(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  if extract_answer(answer, extract_target(data[i]['output'])):
    acc = acc+1
  print(answer)
  print(data[i]['output'])


To answer the above question, the relevant sentence is: Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. So the final answer is .
Answer: from plants
Regulation
Answer: Regulation
Lee surrendered to General Robert E. Lee at Appomattox Court House, Virginia, on June 9, 1865 to end the American Civil War. Therefore, the final answer is .
Answer: Battle of Appomattox Court House
He suffered two major heart attacks (in 1966 and 1969), and was on bed rest for six weeks after the second episode. Therefore, the final answer is major attacks.
Answer: major heart attacks
Red Skelton
Answer: Red Skelton
The town lies along the stretch of coastline designated as Gold Beach during the D-Day landings , one of the beaches used by British troops in the allied invasion. So, the answer is yes.
Answer: no
Hemoglobin molecules that aggregate are called sickle cell trait. People with the trait ha

In [ ]:
print(acc/50)

0.64


In [ ]:
cosine = 0
for i in range(50):
  prompt = zero_shot_CoT(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, data[i]['explanation'])
  cosine = cosine + c
  print(c)
  print(answer)
  print(data[i]['explanation'])


0.8632906675338745
To answer the above question, the relevant sentence is: Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. So the final answer is .
The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of the medicines found in plants mentioned by the author. Thus it can be stated with certainty that some medicines do indeed come from plants.

Therefore, "from plants" is the correct answer option to this question based on the context provided."
0.5346497893333435
Regulation
This act has all the features of Regulation. The act is addressed to an European Union country; it must be applied once and for all within a certain time-limit; it lays down general rules of application, which are binding on all Member States.
0.744847893

In [ ]:
print(cosine/50)

0.5514870642870665


In [ ]:
def zero_shot_ToT(text):
        return f"""
        You are a reasoning assistant. Break down the problem by exploring multiple lines of reasoning. Each thought should follow a different logical path.
        Q: {text}
        Thought 1: Approach the question using common sense reasoning.
        Thought 2: Consider possible assumptions and implications.
        Thought 3: Analyze the question from a cause-effect perspective.
        After exploring all thoughts, decide on the most reasonable final answer.
        A:Answer the question
        """


In [ ]:
acc=0
for i in range(50):
  prompt = zero_shot_ToT(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  if extract_answer(answer, extract_target(data[i]['output'])):
    acc = acc+1
  print(answer)
  print(data[i]['output'])


Thought 1: Approach the question using common sense reasoning.
Answer: from plants
Regulation
Answer: Regulation
Thought 3: Analyze the question from a cause-effect perspective. After exploring all thoughts, decide on the most reasonable final answer.
Answer: Battle of Appomattox Court House
Thought 1: He suffered two major heart attacks
Answer: major heart attacks
Red Skelton
Answer: Red Skelton
Thought 1: The town lies along the stretch of coastline designated as Gold Beach during the D-Day landings , one of the beaches used by British troops in the allied invasion. Thus, the final answer is yes.
Answer: no
Thought 1: Approach the question using common sense reasoning.
Answer: Sue
Thought 1: Consider possible assumptions and implications.
Answer: lips
Thought 3: She stroked Alice's hair and let the child weep. Thus, the final answer is no.
Answer: no
Where can we watch the match Sweden - Germany in Lisbon?
Answer: Where to watch German world cup soccer games in Lisbon?
It's impossibl

In [ ]:
print(acc/50)

0.56


In [ ]:
cosine = 0
for i in range(50):
  prompt = zero_shot_ToT(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, data[i]['explanation'])
  cosine = cosine + c
  print(c)
  print(answer)
  print(data[i]['explanation'])


0.04757644981145859
Thought 1: Approach the question using common sense reasoning.
The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of the medicines found in plants mentioned by the author. Thus it can be stated with certainty that some medicines do indeed come from plants.

Therefore, "from plants" is the correct answer option to this question based on the context provided."
0.5346497893333435
Regulation
This act has all the features of Regulation. The act is addressed to an European Union country; it must be applied once and for all within a certain time-limit; it lays down general rules of application, which are binding on all Member States.
0.03798941150307655
Thought 3: Analyze the question from a cause-effect perspective. After exploring all thoughts, decide on the most reasonable final answer.
The Battle of Appomattox Cour

In [ ]:
print(cosine/50)

0.3659234801866114


Zero-shot + ToT

Zero-shot + CoT + SC

In [ ]:
def zero_shot_CoT(text):
    return f"""
    You are a reasoning assistant. Break down the problem by exploring multiple lines of reasoning.
    Q: {text}. A: Let's think step by step and explain. Answer:...Explanation:...."""

In [ ]:
  def self_consistency(prompt, num_samples=5, max_len=50):
        outputs = []
        for i in range(num_samples):
            input = tokenizer(prompt, return_tensors='pt').to('cuda')
            output = model.generate(
                **input,
                max_length = max_len,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.9,
                num_return_sequences=1
            )
            decoded = tokenizer.decode(output[0], skip_special_tokens=True)
            outputs.append(decoded)
        return outputs

In [ ]:
cosine = 0
def zero_shot_CoT_SC(text, output, num_samples=5, max_len=50, do_print=False):
        prompt = zero_shot_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        c = 0
        if do_print:
          for sample in samples:
            print('Sample : ', sample)
            c = c+ cosine_similarity_score(sample, output)
            print('Cs: ', cosine_similarity_score(sample, output))
            print()
        return samples, c/3

In [ ]:
cosine=0
for i in range(50):
  prompt, c = zero_shot_CoT_SC(data[i]['input'], data[i]['explanation'], num_samples=3, max_len = 300, do_print=True)
  cosine = cosine + c
  print('Target: ', data[i]['output'])
  print('Explanation (target): ', data[i]['explanation'])
  print('--------------------------------------------------------------------')

Sample :  The important part for answering this question is: Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. So the final answer is directly from plants.
Cs:  0.8747879266738892

Sample :  The relevant information is: Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. So the final answer is directly from plants.
Cs:  0.884377121925354

Sample :  Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. So the final answer is plants.
Cs:  0.8542413711547852

Target:  Answer: from plants
Explanation (target):  The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come d

In [ ]:
print(cosine/50)

0.5336776246068379


**Few-shots**

In [ ]:
def few_shots_direct(text):
    return f"""
You are a reasoning assistant. Answer the question directly and clearly, providing a concise justification.

Q: Where do some medicines and recreational drugs come from?
Context: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. [...] Popular stimulants come from plants, such as caffeine from coffee, tea and chocolate, and nicotine from tobacco.
Choices: A. from animals  B. from plants  C. synthetically only  D. from minerals

A: The passage states that substances like caffeine and nicotine come from plants. So the correct answer is B.
Answer: from plants

Q: {text}
A: Answer briefly and justify your choice in one or two sentences.
"""


In [ ]:
acc=0
for i in range(3, 53):
  prompt = few_shots_direct(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  if extract_answer(answer, extract_target(data[i]['output'])):
    acc = acc+1
  print(answer)
  print(data[i]['output'])


major heart attacks
Answer: major heart attacks
Red Skelton
Answer: Red Skelton
It is not clear that the Normandy landings took place in June 1944. So the correct answer is no.
Answer: no
Sue has sickle cell trait, so her hemoglobin molecules aggregate. So the correct answer is Sue.
Answer: Sue
the lips
Answer: lips
No one stroked Mama's hair and let the child weep. So the correct answer is no.
Answer: no
Where can we watch the match Sweden - Germany in Lisbon?
Answer: Where to watch German world cup soccer games in Lisbon?
Yes
Answer: Yes
E.
Answer: E
0
Answer: 0
Yes
Answer: Yes
I want an extra cheesy buffalo pizza.
Answer: Great I just got it! Says it will be here in 35 minutes. Thank you so much! You have a great day!
B.
Answer: B
Hieronymus
Answer: Hieronymus
Ben had normal vision. So the correct answer is Ben.
Answer: Ben
Hunger is an effective motivator for beginning work. Begin work is the action of starting something. So the correct answer is begin work
Answer: begin work
Harol

In [ ]:
print(acc/50)

0.66


In [ ]:
cosine = 0
for i in range(50):
  prompt = few_shots_direct(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, data[i]['explanation'])
  cosine = cosine + c
  print(c)
  print(answer)
  print(data[i]['explanation'])


0.0665941908955574
B.
The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of the medicines found in plants mentioned by the author. Thus it can be stated with certainty that some medicines do indeed come from plants.

Therefore, "from plants" is the correct answer option to this question based on the context provided."
0.5346497893333435
Regulation
This act has all the features of Regulation. The act is addressed to an European Union country; it must be applied once and for all within a certain time-limit; it lays down general rules of application, which are binding on all Member States.
0.6499089002609253
The relevant information to answer the above question is: Robert E. Lee surrendered to Ulysses S. Grant at Appomattox Court House, Virginia, on June 9, 1865, ending the American Civil War. So the correct answer is Arlington Nation

In [ ]:
print(cosine/50)

0.4717605393566191


Few-shots + CoT

In [6]:
def few_shots_CoT(text):
    return f"""
You are a reasoning assistant. For each question, think through the context step by step before answering. Do not jump to the final answer without explanation.

Q: Where do some medicines and recreational drugs come from?
Context: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. [...] Popular stimulants come from plants, such as caffeine from coffee, tea and chocolate, and nicotine from tobacco.
Choices:
A. from animals
B. from plants
C. synthetically only
D. from minerals

A: Let's think step by step.
The context mentions that popular stimulants like caffeine and nicotine are found in plants like coffee and tobacco.
It also states that aspirin is derived from willow bark, which is also a plant.
There is no mention of animals, minerals, or synthetic-only origins.
So the best answer is B.
Answer: from plants

---

Q: Why do metal objects feel colder than wooden ones at the same temperature?
Context: Even when objects are at the same temperature, materials conduct heat differently. Metals are good conductors of heat, while wood is a poor conductor.
Choices:
A. Because metals are colder
B. Because wood generates heat
C. Because metals conduct heat away from the skin faster
D. Because wood holds heat longer

A: Let's think step by step.
When you touch an object, heat transfers from your skin to the object if it's cooler.
Metals conduct heat much better than wood, so they pull heat from your skin more quickly.
This makes them feel colder, even though both objects are the same temperature.
Therefore, the correct answer is C.
Answer: Because metals conduct heat away from the skin faster

---

Q: {text}
A: Let's think step by step.Explanation... Always explain your reasoning step-by-step, even if the answer seems obvious.
You must not provide the final answer until you have explained your reasoning step by step.


"""


In [ ]:
clear_gpu()

In [12]:
acc=0
cosine = 0
for i in range(3, 53):
  prompt = few_shots_CoT(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, data[i]['explanation'])
  cosine = cosine + c
  print(c)
  print('Answer: ', answer)
  print('Explanation (target): ', data[i]['explanation'])
  if extract_answer(answer, extract_target(data[i]['output'])):
    acc = acc+1
  print('Target: ', data[i]['output'])
  print('------------------------------------------------------------')


0.3914015591144562
Answer:  major heart attacks ---
Explanation (target):  The answer is the number of major heart attacks.
Though he was on bed rest for six weeks after his second attack, that's not greater than the total number of times (i.e., two) Nasser suffered from a heart attack.
Target:  Answer: major heart attacks
------------------------------------------------------------
0.5166747570037842
Answer:  Red Skelton ---
Explanation (target):  The text is about Red Skelton. The context describes events and details of his life, such as how he met Edna Stillwell in Vincennes and their early careers, the "Doughnut Dunkers" routine that brought him recognition, his performances at Loew's State Theatre in New York City with Edna (where they were booked by Harry Anger) which led to them performing for President Roosevelt later on.
So going through all these details it can clearly be concluded that this passage is only relevant to one person i.e., Richard ""Red"" Skelton himself."
Target

In [16]:
print(acc/50)
print(cosine/50)

0.0
0.0


Few-shots + CoT + SC

In [ ]:
clear_gpu()

In [ ]:
cosine = 0
def few_shots_CoT_SC(text, output, num_samples=5, max_len=50, do_print=False):
        prompt = few_shots_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        c = 0
        if do_print:
          for sample in samples:
            print('Sample : ', sample)
            c = c+ cosine_similarity_score(sample, output)
            print('Cs: ', cosine_similarity_score(sample, output))
            print()
        return samples, c/3

In [ ]:
cosine=0
for i in range(50):
  prompt, c = few_shots_CoT_SC(data[i]['input'], data[i]['explanation'], num_samples=3, max_len = 300, do_print=True)
  cosine = cosine + c
  print('Target: ', data[i]['output'])
  print('Explanation (target): ', data[i]['explanation'])
  print('--------------------------------------------------------------------')

Sample :  The context mentions that popular stimulants like caffeine and nicotine are found in plants like coffee and tobacco. It also states that aspirin is derived from willow bark, which is also a plant. There is no mention of animals, minerals, or synthetic-only origins. So the best answer is B. Answer: from plants ---
Cs:  0.6532799005508423

Sample :  B. Answer: from plants ---
Cs:  0.511865496635437

Sample :  B. Answer: from plants ---
Cs:  0.511865496635437

Target:  Answer: from plants
Explanation (target):  The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of the medicines found in plants mentioned by the author. Thus it can be stated with certainty that some medicines do indeed come from plants.

Therefore, "from plants" is the correct answer option to this question based on the context provided."
---------------------

In [26]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
task_samples = data[:350]
new_tasks = []
for task in task_samples:
    task["chain_of_thought"] = task['explanation']
    task["answer"] = task['output']
    new_tasks.append(task)
texts = [task['input'] for task in new_tasks]
vectorizer = TfidfVectorizer().fit(texts)
X = vectorizer.transform(texts)
def find_top_k_tasks(text, k=3):
        q = vectorizer.transform([text])
        scores = cosine_similarity(q, X).flatten()
        top = scores.argsort()[::-1][:k]
        return [new_tasks[i] for i in top]
def find_similar_task(query):
    q = vectorizer.transform([query])
    idx = cosine_similarity(q, X).argmax()
    return new_tasks[idx]
test_tasks = data[350:]

In [27]:
  def build_prompt(examples, query):
        prompt = ""
        for ex in examples:
            prompt += f"""
            Task: Reasoning for text
            Input: {ex['input']}
            Output: {ex['output']}
            """
            prompt += (
                "Now solve this task:\n"
                f"Input: {query}\n"
                "Solution:\n"
                "Let's solve this step-by-step. Write your reasoning clearly."
            )
        return prompt

In [58]:
clear_gpu()

In [28]:
len(test_tasks)

53

In [56]:
def few_shots_CoT_ART(text, k=1):
        examples = find_top_k_tasks(text, k)
        return build_prompt(examples, text)
acc=0
for i in range(50):
  prompt = few_shots_CoT_ART(test_tasks[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input, max_len=200)[0], skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, test_tasks[i]['explanation'])
  cosine = cosine + c
  print(c)
  print('Answer: ', answer)
  print('Explanation (target): ', test_tasks[i]['explanation'])
  if extract_answer(answer, extract_target(test_tasks[i]['output'])):
    acc = acc+1
  print('Target: ', test_tasks[i]['output'])
  print('------------------------------------------------------------')


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.51 GiB. GPU 0 has a total capacity of 14.74 GiB of which 3.08 GiB is free. Process 7085 has 11.66 GiB memory in use. Of the allocated memory 11.50 GiB is allocated by PyTorch, and 29.70 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [61]:
for i in range(3, 53):
  prompt = few_shots_CoT_ART(data[i]['input'],3)
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  answer = tokenizer.decode(generate_output(type=None, input = input)[0], skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, data[i]['explanation'])
  cosine = cosine + c
  print(c)
  print('Answer: ', answer)
  print('Explanation (target): ', data[i]['explanation'])
  if extract_answer(answer, extract_target(data[i]['output'])):
    acc = acc+1
  print('Target: ', data[i]['output'])
  print('------------------------------------------------------------')

OutOfMemoryError: CUDA out of memory. Tried to allocate 654.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 384.12 MiB is free. Process 7085 has 14.36 GiB memory in use. Of the allocated memory 14.16 GiB is allocated by PyTorch, and 83.61 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Few-shots ToT

In [44]:
import re

def get_best_thought(thoughts, text, do_print=False):
    output = select_best_path(thoughts, text, do_print=do_print)
    if 0<len(output) < 3:
        match = re.search(r"\b(\d+)\b", output)
        if match:
          index = int(match.group(1)) - 1
          if 0 <= index < len(thoughts):
              return thoughts[index]
    else:
        return output

In [45]:
def expand_thoughts(prompt, n=3):
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = [model.generate(**inputs, max_length=150) for _ in range(n)]
    return [tokenizer.decode(o[0], skip_special_tokens=True) for o in outputs]

def recursive_expand_tree(prompt, depth, breadth, context, do_print=False):
    if depth == 0:
        return [prompt]

    base_prompt = f"""
        You are a reasoning assistant. Use tree-of-thought reasoning to explore solutions.

        Question: {context}

        Partial Reasoning:
        {prompt}
        Now expand with next steps:
        """

    expanded = expand_thoughts(base_prompt, n=breadth)
    expanded = list(set(expanded))
    if do_print:
        print(f"[Depth {depth}] Base prompt:\n{base_prompt}")
        print(f"[Depth {depth}] Got thoughts:\n", expanded)

    tree = []
    for thought in expanded:
        sub_tree = recursive_expand_tree(thought, depth - 1, breadth, context, do_print)
        tree.extend(sub_tree)
    return tree


def select_best_path(thoughts, text, do_print=False):
    prompt = f"""You are a reasoning expert. Given a question and several reasoning paths, choose the most logical one.

    Question: {text}

    Candidates:
    {chr(10).join([f"{i + 1}. {t}" for i, t in enumerate(thoughts)])}

    Reply with the best option, justify briefly. Final answer is ?"""
    if do_print:
            print('prompt:\n', prompt)
    print(thoughts)
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(**inputs, max_length=300)
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()

def zero_shot_ToT_expanded(text, depth=2, breadth=3, do_print=False):
    root_prompt = zero_shot_ToT(text)
    if do_print:
            print('Root_prompt:\n', root_prompt)
    tree = recursive_expand_tree(root_prompt, depth, breadth, text, do_print)
    best = get_best_thought(tree, text, do_print)
    return best
def few_shots_ToT_expanded(text, depth=2, breadth=3, do_print=False):
    root_prompt = few_shots_ToT(text)

    if do_print:
        print('[Root prompt]')
        print(root_prompt)

    tree = []
    for _ in range(breadth):
        single_tree = recursive_expand_tree(root_prompt, depth, breadth, text, do_print)
        tree.extend(single_tree)

    best_path = get_best_thought(tree, text, do_print)
    return best_path


In [46]:
clear_gpu()

In [ ]:
acc = 0
cosine = 0
for i in range(50):
  answer = zero_shot_ToT_expanded(data[i]['input'], depth=2, breadth=2, do_print=False)
  if answer != None:
    if extract_answer(answer,data[i]['output']):
      acc = acc + 1
    c = cosine_similarity_score(answer, data[i]['explanation'])
    print('CS: ', c)
    cosine = cosine + c
  print('Answer: ', answer)
  print('Target: ', data[i]['explanation'])
  print('Output: ', data[i]['output'])

['Thought 1: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. Others, such as the essential oils peppermint oil and lemon oil are useful for their aroma, as flavourings and spices (e.g., capsaicin), and in medicine as pharmaceuticals as in opium from opium poppies. Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. Others are simple derivatives of botanical natural products. For example, the pain killer aspirin is the acetyl este']
CS:  0.6236366033554077
Answer:  Thought 1: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. Others, such as the essential oils peppermint oil and lemon oil are useful for their aroma, as flavourings and spices (e.g., capsaicin), and in medicine as pharmaceutic

In [ ]:
print(acc/50)
print(cosine/50)

0.0
0.4122199112176895


In [48]:
def few_shots_ToT(text):
    return f"""
You are a reasoning assistant. For each question, explore multiple lines of reasoning. Each thought should follow a different logical path. You must explain clearly in each path. Only after that, you give the final answer.

Q: Where do some medicines and recreational drugs come from?
Context: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. [...] Popular stimulants come from plants, such as caffeine from coffee, tea and chocolate, and nicotine from tobacco.
Choices:
A. from animals
B. from plants
C. synthetically only
D. from minerals

Thought 1 (Common sense reasoning):
We commonly know that many drugs like caffeine, nicotine, and morphine are derived from natural sources like tea, tobacco, and poppy plants. These are familiar examples that most people recognize as plant-derived substances.

Thought 2 (Assumptions and implications):
The question implies a general source for medicines and recreational drugs. The context gives multiple examples of plant-based compounds, and no examples from animals or synthetic-only processes. So it's implied that plants are a primary source. If the answer were "animals" or "synthetic only," the text would need to mention them, but it does not.

Thought 3 (Cause-effect analysis):
Plants undergo secondary metabolism which produces chemical compounds like alkaloids. These have strong physiological effects on humans (e.g., caffeine stimulates the nervous system). That biological capability explains why plants are a common origin of such drugs.

Final Answer: from plants

---

Q: Why do metal objects feel colder than wooden ones at the same temperature?
Context: Even when objects are at the same temperature, materials conduct heat differently. Metals are good conductors of heat, while wood is a poor conductor.
Choices:
A. Because metals are colder
B. Because wood generates heat
C. Because metals conduct heat away from the skin faster
D. Because wood holds heat longer

Thought 1 (Common sense reasoning):
Touching a metal door handle feels much colder than touching a wooden table, even if both have been in the same room. This everyday experience suggests metal pulls heat away from us more quickly.

Thought 2 (Assumptions and implications):
The assumption is that both materials are at the same temperature, so the cause of the sensation must be related to heat transfer, not temperature. The implication is that perception of "coldness" comes from heat leaving our body faster when we touch metal.

Thought 3 (Cause-effect analysis):
Metals are thermal conductors — they rapidly transfer heat from our warm skin into the cooler material. Wood is an insulator, so the heat stays near the contact area and doesn't flow away as fast, making it feel warmer to touch.

Final Answer: Because metals conduct heat away from the skin faster

---

Q: {text}
Thought 1 (Common sense reasoning):
Thought 2 (Assumptions and implications):
Thought 3 (Cause-effect analysis):

Final Answer:
"""


In [50]:
acc = 0
cosine = 0
for i in range(50):
  prompt = few_shots_ToT(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  output = generate_output(input=input, type=None)
  answer = tokenizer.decode(output[0], skip_special_tokens=True).strip()
  if answer != None:
    if extract_answer(answer,data[i]['output']):
      acc = acc + 1
    c = cosine_similarity_score(answer, data[i]['explanation'])
    print('CS: ', c)
    cosine = cosine + c
  print('Answer: ', answer)
  print('Target: ', data[i]['explanation'])
  print('Output: ', data[i]['output'])

CS:  0.5821842551231384
Answer:  from plants ---
Target:  The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of the medicines found in plants mentioned by the author. Thus it can be stated with certainty that some medicines do indeed come from plants.

Therefore, "from plants" is the correct answer option to this question based on the context provided."
Output:  Answer: from plants
CS:  0.48955485224723816
Answer:  Regulation ---
Target:  This act has all the features of Regulation. The act is addressed to an European Union country; it must be applied once and for all within a certain time-limit; it lays down general rules of application, which are binding on all Member States.
Output:  Answer: Regulation
CS:  0.7139257192611694
Answer:  Thought 3 (Cause-effect analysis): Lee surrendered to General Robert E. Lee at Appomattox Court

In [51]:
print(acc/50)
print(cosine/50)

0.0
0.436635173112154


In [ ]:
acc = 0
cosine = 0
for i in range(50):
  answer = few_shots_ToT_expanded(data[i]['input'], depth=2, breadth=2, do_print=False)
  if answer != None:
    if extract_answer(answer,data[i]['output']):
      acc = acc + 1
    c = cosine_similarity_score(answer, data[i]['explanation'])
    print('CS: ', c)
    cosine = cosine + c
  print('Answer: ', answer)
  print('Target: ', data[i]['explanation'])
  print('Output: ', data[i]['output'])

['The important part for answering this question is: Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. So the final answer is from plants.', 'The important part for answering this question is: Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. So the final answer is from plants.']
CS:  0.8621091246604919
Answer:  The important part for answering this question is: Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. So the final answer is from plants.
Target:  The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of

In [ ]:
print(cosine/50)

0.46631482064723967


# Finetune

In [ ]:
import json
classification = []
with open('classification_instruction_data_en.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        classification.append(json.loads(line.strip()))
qa = []
computation = []
reasoning = []

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
os.environ["WANDB_DISABLED"] = "true"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model2 = T5ForConditionalGeneration.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False
)
trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model2)
)
trainer.train()